In [20]:
import nibabel as nib
import numpy as np
import imageio
import os
def makedir(path):
    if os.path.exists(path):
        pass
    else:
        os.mkdir(path)

def read_niifile(niifile):  # 读取niifile文件
    img = nib.load(niifile)  # 下载niifile文件（其实是提取文件）
    img_fdata = img.get_fdata()  # 获取niifile数据
    return img_fdata


def save_fig(file,savepicdir,name):  # 保存为图片
    fdata = read_niifile(file)  # 调用上面的函数，获得数据
    # (x, y, z) = fdata.shape  # 获得数据shape信息：（长，宽，维度-切片数量）
    # for k in range(z):
    silce = fdata[:, :, i]  # 三个位置表示三个不同角度的切片
    imageio.imwrite(os.path.join(savepicdir, '{}.png'.format(name)), silce)
        # 将切片信息保存为png格式
def getName(na):
    for i in range(0,len(na)):
        if na[i]=='_':
            return na[:i]
#传入nii.gz文件所在文件夹
niiGzFolder=r'C:/Users/赵/Desktop/2023fallterm/IME/section 2/PROJECT2/COVID-19-CT-Seg_20cases'
niiGzNames=os.listdir(niiGzFolder)
#存放解压后的分割图片
target = r'C:/Users/赵/Desktop/2023fallterm/IME/section 2/PROJECT2/phaoto'
for i in range(0,len(niiGzNames)):
    niiGzName=niiGzNames[i]
    name=getName(niiGzName)
    pred="pred"
    gt="gt"
    img="img"
    if  pred in niiGzName:
        path=target+pred+'/'

    if gt in niiGzName:
        path=target+gt+'/'

    if img in niiGzName:
        path=target+img+'/'
    makedir(path)
    save_fig(niiGzFolder+niiGzName,path,name)
    print("finish",niiGzName)

NameError: name 'path' is not defined

In [ ]:
import torch
import cv2
import os
import glob
from torch.utils.data import Dataset
import random


class Loader(Dataset):
    def __init__(self, data_path):
        # print(data_path)
        images = []
        image_root = os.path.join(data_path,'images')
        # print(image_root)
        for image_name in os.listdir(image_root):
          image_path = os.path.join(image_root, image_name)
          images.append(image_path)
        # 初始化函数，读取所有data_path下的图片
        self.data_path = data_path
        
        self.imgs_path = images

    def augment(self, image, flipCode):
        # 使用cv2.flip进行数据增强，filpCode为1水平翻转，0垂直翻转，-1水平+垂直翻转
        flip = cv2.flip(image, flipCode)
        return flip

    def __getitem__(self, index):
        # 根据index读取图片
        image_path = self.imgs_path[index]
        # 根据image_path生成label_path
        label_path = image_path.replace('images', 'labels')
        # 读取训练图片和标签图片
        image = cv2.imread(image_path)
        label = cv2.imread(label_path)
        # 将数据转为单通道的图片
        image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        label = cv2.cvtColor(label, cv2.COLOR_BGR2GRAY)
        image = image.reshape(1, image.shape[0], image.shape[1])
        label = label.reshape(1, label.shape[0], label.shape[1])
        # 处理标签，将像素值为255的改为1
        if label.max() > 1:
            label = label / 255
        # 随机进行数据增强，为2时不做处理
        flipCode = random.choice([-1, 0, 1, 2])
        if flipCode != 2:
            image = self.augment(image, flipCode)
            label = self.augment(label, flipCode)
        return image, label

    def __len__(self):
        # 返回训练集大小
        return len(self.imgs_path)


if __name__ == "__main__":
    path= r'C:\Users\赵\Desktop\2023fallterm\IME\section 2\PROJECT2\phaoto\\'
    isbi_dataset = Loader(path)
    print("数据个数：", len(isbi_dataset))
    train_loader = torch.utils.data.DataLoader(dataset=isbi_dataset,
                                               batch_size=2,
                                               shuffle=True)
    for image, label in train_loader:
        print(image.shape)